In [60]:
import os
import cv2
import numpy as np

In [61]:
face_model = 'models/opencv_face_detector_uint8.pb'
face_proto = 'models/opencv_face_detector.pbtxt'
age_model = 'models/age_net.caffemodel'
age_proto = 'models/age_deploy.prototxt'
gender_model = 'models/gender_net.caffemodel'
gender_proto = 'models/gender_deploy.prototxt'

In [62]:
face_net = cv2.dnn.readNet(face_model, face_proto)
age_net = cv2.dnn.readNet(age_model, age_proto)
gender_net = cv2.dnn.readNet(gender_model, gender_proto)

In [63]:
AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
GENDER_LIST = ['Male', 'Female']

In [64]:
folder_path = 'Task2_img'

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.webp'):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        if image is None:
            print(f"Error: Unable to read image '{image_path}'. Skipping...")
            continue

        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)

        # Detect faces
        face_net.setInput(blob)
        detections = face_net.forward()

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                face = image[startY:endY, startX:endX]
                face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), [78.4263377603, 87.7689143744, 114.895847746], swapRB=False)

                # Predict gender
                gender_net.setInput(face_blob)
                gender_preds = gender_net.forward()
                gender = GENDER_LIST[gender_preds[0].argmax()]

                # Predict age
                age_net.setInput(face_blob)
                age_preds = age_net.forward()
                age = AGE_LIST[age_preds[0].argmax()]

                # Draw the bounding box and label on the image
                label = f'{gender}, {age}'
                cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
                cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

        resized_image = cv2.resize(image, (850, 900))  # Adjust the size as needed

        # Display the output image
        cv2.imshow('Output', resized_image)
        cv2.waitKey(0)

In [65]:
cv2.destroyAllWindows()